In [1]:
import pandas as pd
from typing import List

In [37]:
def balance_df(df, columns:List, rs=42):
    # minimum count of rows for any combination of values in the specified columns
    min_count = df.groupby(columns).size().min()
    
    # sample min_count rows for each column combination
    balanced_df = (
        df.groupby(columns)[df.columns]
          .apply(lambda group: group.sample(min_count, random_state=rs))
          .reset_index(drop=True)
    )
    
    return balanced_df.sample(frac=1).reset_index(drop=True)    # shuffle the rows

In [ ]:
# code used to create the data splits

# merging the proposals set with the different subsets (train, valid, test)
# CFS = pd.read_csv('../data/CoFE/CFS_without_debates_CFET.tsv', sep='\t')
# proposals = pd.read_csv('../data/CoFE/prop_CF_all.tsv', sep='\t')
# CoFE_train = pd.merge(proposals, CFS.drop('Topic', axis=1), left_on='id', right_on='id_prop', suffixes=('_prop', '_cm'))
# CoFE_train = CoFE_train.loc[:, ~CoFE_train.columns.duplicated()]
# CoFE_train.to_csv('../data/CoFE/CoFE_train.csv', index=None)

# keeping only samples where both the target and the comment are in either German, French or English (train set only)
# (to avoid balancing with rare languages and dropping too much data)
# CoFE_train_filtered = CoFE_train[CoFE_train['lan_prop'].isin(['de', 'fr', 'en']) & CoFE_train['lan_cm'].isin(['de', 'fr', 'en'])]

# adding a label column that is exactly the same as the alignment column for consistency during training (train set only)
# CoFE_train_filtered['label'] = CoFE_train_filtered['alignment']
# CoFE_train_filtered.to_csv('../data/CoFE/CoFE_train_filtered.csv', index=None)

# dropping the 'Other' label for consistency with the train set (valid set only)
# valid = pd.read_csv('../data/CoFE/CoFE_valid.csv')
# valid_filtered = valid[~(valid['label'] == 'Other')]
# valid_filtered.to_csv('../data/CoFE/CoFE_valid_filtered.csv', index=None)